# 02 - GEDI L2A: test case - Mangroves

<div class="alert alert-warning">

**GEDI-2A DATA DICTIONARY**:

Een referentielijst met alle terminologie van de GEDI-Level 2A shots is te vinden op: [https://lpdaac.usgs.gov/documents/586/gedi_l2a_dictionary_P001_v1.html](https://lpdaac.usgs.gov/documents/586/gedi_l2a_dictionary_P001_v1.html) 

In onderstaande voorbeelden gaan we dieper in op de visualisatie van een GEDI-Level-2A shots voor het mangrovebos in Saramacca.

### 1. Inladen packages

In [ ]:
import os
import h5py
import numpy as np
import pandas as pd
import geopandas as gp
from shapely.geometry import Point
import geoviews as gv
from geoviews import opts, tile_sources as gvts
import holoviews as hv
gv.extension('bokeh', 'matplotlib')

In [ ]:
# Volgende code maakt een link aan van je datafolder, waar je gedownloade GEDI-datafile zou moeten staan
dataDir = os.getcwd() + os.sep + 'data' + os.sep

In [ ]:
dataDir

In [ ]:
gediFiles = [g for g in os.listdir(dataDir) if g.startswith('GEDI02_A') and g.endswith('.h5')]  # List all GEDI L1B .h5 files in inDir
gediFiles

Hier zie en onmiddellijk het formaat van een GEDI-dataset: 

> **GEDI02_A**: Naam van GEDI product
**2019170155833**: Datum en tijd van opname volgens Juliaanse notatie (YYYYDDDHHMMSS)  
**O02932**: Orbit Number
**02**: Sub-Orbit Granule Number (1-4)  
**T02267**: Track Number (Reference Ground Track)  
**02**: Positioning and Pointing Determination System (PPDS) type (00 is predict, 01 rapid, 02 and higher is final)  
**003**: PGE Version Number  
**01**: Granule Production Version  
**V002**: Product Version

In [ ]:
L2A = gediFiles[0]

### Inlezen van een HDF5-file

In [ ]:
gediL2A = h5py.File('data'+"\\" + L2A,'r')

### Bekijken van aanwezige *keys*

In [ ]:
beamNames = list(gediL2A.keys())
beamNames

Uit de lijst hierboven zie je dat de dataset is onderverdeeld in 8 verschillende 'BEAMS' + een METADATA-file.  

Dit is omwille van het feit data GEDI in totaal 8 'shots' maakt per moment, over afstand van 4.2 km.

Hiervan zijn de helft van de BEAM's *full power* beams, en de helft *coverage* beams. Het verschil is dat de 'coverage' beams slechts tot 95% in het kronendak penetreren, terwijl de full beams sterkere shots zijn. Zeker voor dense bossen zijn deze type beams niet bruikbaar. Doorgaans wordt aangeraden enkel met de *full power* beams te werken.

![img/GEDI_BEAMS.jpg](img\GEDI_BEAMS.jpg)

Verder kunnen we bekijken welke informatie opgeslagen zit in de metadata:

In [ ]:
for g in gediL2A['METADATA']['DatasetIdentification'].attrs: print(g) 

In [ ]:
print(gediL2A['METADATA']['DatasetIdentification'].attrs['purpose'])


We kunnen de beamnames ook bekijken:

In [ ]:
#Zorgen dat enkel de elementen met 'BEAM' overblijven in de lijst
beamNames = [g for g in gediL2A.keys() if g.startswith('BEAM')]
beamNames

In [ ]:
for b in beamNames: 
    print(f"{b} is a {gediL2A[b].attrs['description']}")
    print(f"{b} has as number {np.unique(gediL2A[b+ '/beam'][()])[0]}")

Vervolgens pikken we hier eentje uit, om verder na te gaan wat in elke BEAM zit opgeslagen

In [ ]:
beamNames

In [ ]:
gediL2A_objs = []
gediL2A.visit(gediL2A_objs.append)                                           # Retrieve list of datasets
gediSDS = [o for o in gediL2A_objs if isinstance(gediL2A[o], h5py.Dataset)]  # Search for relevant SDS inside data file
[i for i in gediSDS if 'BEAM1000' in i][0:10]

## 4. Visualizeren van een GEDI Orbit

In onderstaande code halen we de geografische  informatie uit de GEDI SDS (=Science DataSet), wat per beam opgeslagen zit. 

In [ ]:
lonSample, latSample, shotSample, qualitySample, beamSample = [], [], [], [], []  # Set up lists to store data

# Open the SDS
lats = gediL2A['BEAM1000/lat_lowestmode'][()]
lons = gediL2A['BEAM1000/lon_lowestmode'][()]
shots = gediL2A['BEAM1000/shot_number'][()]
quality = gediL2A['BEAM1000/quality_flag'][()]

# Take every 10th shot and append to list
for i in range(len(shots)):
    if i % 10 == 0:
        shotSample.append(str(shots[i]))
        lonSample.append(lons[i])
        latSample.append(lats[i])
        qualitySample.append(quality[i])
        beamSample.append('Beam1000')

# Write all of the sample shots to a dataframeb
latslons = pd.DataFrame({'Beam': beamSample, 'Shot Number': shotSample, 'Longitude': lonSample, 'Latitude': latSample,
                         'Quality Flag': qualitySample})
latslons

In [ ]:
#Sommige variabelen hebben we niet meer nodig, dus verwijderen we die:
del beamSample, quality, qualitySample, gediL2A_objs, latSample, lats, lonSample, lons, shotSample, shots 

Met Geopandas kunnen we een GeoDataFrame aanmaken, op basis van de Longitude en Latitude kolommen:

In [ ]:
latslons['geometry'] = latslons.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)

In [ ]:
type(latslons)

In [ ]:
#O Omzetten naar een GeoDataFrame
latslons = gp.GeoDataFrame(latslons)
latslons = latslons.drop(columns=['Latitude','Longitude'])
latslons['geometry']

In [ ]:
latslons.crs

Er is nog geen CRS toegekend, maar we weten dat deze EPSG=4326 heeft, onderstaande code past dit aan:

In [ ]:
latslons = latslons.set_crs(epsg=4326)

In [ ]:
#Hier kunnen we weer een punt bekijken
latslons['geometry'][0]


In [ ]:
###Visualizeren van de Volledige GeoDataFrame:
ax = latslons.explore()

Bovenstaande zegt ons nog niet zoveel, daarom gaan we via een extra plotmethode ook een achtergrond toevoegen:

In [ ]:
# Define a function for visualizing GEDI points
def pointVisual(features, vdims):
    return (gvts.EsriImagery * gv.Points(features, vdims=vdims).options(tools=['hover'], height=500, width=900, size=5, 
                                                                        color='yellow', fontsize={'xticks': 10, 'yticks': 10, 
                                                                                                  'xlabel':16, 'ylabel': 16})) 


# Inlezen van Mangrove-ROI

In [ ]:
Mangrove_ROI = gp.read_file('data/ROI_Mangroves.zip')

In [ ]:
Mangrove_ROI = Mangrove_ROI.to_crs(latslons.crs)

In [ ]:
# Create a list of geodataframe columns to be included as attributes in the output map
vdims = []
for f in latslons:
    if f not in ['geometry']:
        vdims.append(f)
vdims

In [ ]:
gv.Polygons(Mangrove_ROI['geometry']).opts(line_color='red', color=None) * pointVisual(latslons, vdims = vdims)

Bij het bekijken van de data, zie je 'Quality Flag' met een 0 of een 1. We kunnen nagaan wat dit precies betekent:

In [ ]:
print(f"Quality Flag: {gediL2A[b]['quality_flag'].attrs['description']}")


# GEDI-L2A data: RH-metrics

GEDI-L2A levert de Relative Height-metrieken, wat een typische LiDAR-metrieken zijn. Deze gaan van RH_0 tot RH_100, waarbij:

* **RH 100** = elev_highestreturn - elev_lowestmode.
              Dit is het verschil tussen de grootste hoogte waarop een 'return'-waarde werd geregistreerd en de vooropgestelde hoogte van het laagste punt (= de grond). Deze hoogte-waarden worden relatief gemeten, maar om dit naar absolute getallen om te zetten maakt GEDI gebruik van het WGS-hoogtemodel. 

* **RH_0** = Hoogte van de laagste return = bodem

* De lagere RH-waarden kunnen **negatieve** waarden hebben, omdat vaak een groter % van de teruggekeerde energie van de grond komt

## Specifieke shots inladen

Eerst kijken we hoeveel unieke shots aanwezig zijn in onze file:

In [ ]:
# Hoeveel unieke shots zitten in onze file?
len(gediSDS)

In [ ]:
beamSDS = [g for g in gediSDS if 'BEAM1000' in g]  # Subset to a single beam
len(beamSDS)

Hieruit gaan we één shot halen, ter illustratie van hoe dergelijke return er uit ziet. Dit doen we op basis van het unieke *shotnummer*. 

Hier kiezen we voor een shot die binnen de mangrove-zone valt. (vooraf geselecteerd via bovenstaande figuren).

In [ ]:
shot = 155570800200091194

In [ ]:
index = np.where(gediL2A['BEAM1000/shot_number'][()]==shot)[0][0]  # Set the index for the shot identified above
index

In [ ]:
rh = gediL2A[[g for g in beamSDS if g.endswith('/rh')][0]]  # Relative Height Metrics

Print de omschrijving van de rh dataset, om hier inzicht in te krijgen:

In [ ]:
print(f"rh is {rh.attrs['description']}")

In [ ]:
algo = gediL2A[f'{beamNames[0]}/selected_algorithm']  # selected algorithm
print(f"selected_algorithm is {algo.attrs['description']}")

Laden van de latitude/longitude van het geselecteerde shot

In [ ]:
# Bring in the desired SDS
lats = gediL2A[f'{beamNames[0]}/lat_lowestmode']  # Latitude
lons = gediL2A[f'{beamNames[0]}/lon_lowestmode']  # Longitude

In [ ]:
rhLat = lats[index]
rhLon = lons[index]
rhShot1 = rh[index]
algoShot1 = algo[index]

In [ ]:
print(f"The shot is located at: {str(rhLat)}, {str(rhLon)} (shot ID: {shot}, index {index}) and is from beam {beamNames[5]}.")
print(f"The selected algorithm is Algorithm Setting Group {str(algoShot1)}.")

In [ ]:
# Grab the elevation recorded at the start and end of the RH metrics
zElevation = gediL2A[[g for g in beamSDS if g.endswith('/elev_lowestmode')][0]][index]  # Elevation
zTop = gediL2A[[g for g in beamSDS if g.endswith('/elev_highestreturn')][0]][index]     # Elevation Highest Return

In [ ]:
rhShot = [z + zElevation for z in rhShot1]  # To convert canopy height to canopy elevation, add the elevation to each value
rh25 = rhShot[24]                           # 25% 
rh50 = rhShot[49]                           # 50%  
rh75 = rhShot[74]                           # 75% 

In [ ]:
rhVis = hv.Curve(rhShot, label=f'Selected Algorithm (a{str(algoShot1)})')
rhVis = rhVis.opts(color='black', tools=['hover'], height=500, width=400, title='GEDI L2A Relative Height Metrics', 
                   xlabel='Percent Energy Returned', ylabel='Elevation (m)', xlim=(0,100),ylim=(np.min(rhShot),np.max(rhShot)), 
                   fontsize={'title':14, 'xlabel':16, 'ylabel': 16, 'legend': 14, 'xticks':12, 'yticks':12}, line_width=3.5)
rhVis


In [ ]:
# Create plots for L2A Metrics
zX = [0,100]                   # set up list from 0 to 100 to create the line
zY = [zElevation, zElevation]  # ground elevation
zT = [zTop, zTop]              # highest return

# Set up plots for each of the desired values
zVis = hv.Curve((zX, zY), label='Ground Return').opts(color='saddlebrown', tools=['hover'], height=550, width=400, line_width=2)
ztVis = hv.Curve((zX, zT), label='RH100').opts(color='navy', tools=['hover'], height=550, width=400, line_width=2)
rh25Vis = hv.Curve((zX, [rh25,rh25]),label='RH25').opts(color='lightblue',tools=['hover'], height=550, width=400, line_width=2)
rh50Vis = hv.Curve((zX, [rh50,rh50]),label='RH50').opts(color='mediumblue',tools=['hover'], height=550, width=400, line_width=2)
rh75Vis = hv.Curve((zX, [rh75,rh75]),label='RH75').opts(color='darkblue',tools=['hover'], height=550, width=400, line_width=2)


In [ ]:
# Plot all of the metrics together
l2aVis = rhVis * zVis * ztVis * rh25Vis * rh50Vis * rh75Vis
l2aVis_backup = l2aVis
l2aVis.opts(show_legend=True, legend_position='bottom_right', title='GEDI L2A Relative Height Metrics', ylabel='Elevation (m)',
                   xlabel='Percent Energy Returned', xlim=(0, 100), ylim=(np.min(rhShot) + 1.5, np.max(rhShot) + 5), height=600,
                   fontsize={'title':16, 'xlabel':16, 'ylabel': 16, 'legend': 14, 'xticks':12, 'yticks':12}, width=400)


<div class="alert alert-success">

**OEFENING**:

Identificeer nu een 2e shotnummer, op basis van de overzichtsfiguur die we hoger hebben aangemaakt. Je kan hierbij spelen met een ander vegetatietype, ...

Maak, op basis van het shotnummer, een gelijkaardige Relative Height-figuur.

# 2 - Visualisatie van L1B

Net zoals GEDI-L2A kan de L1B-data gedownload worden via [NASA Earthdata Search](https://search.earthdata.nasa.gov/search).
Aangezien het hier de ruwe data betreft, en deze files tot een paar GB in grootte kunnen gaan, bekijken we rechtstreeks enkele 'waveforms'.

Voor verdere introductie tot Level 1B kan ik volgende tutorial aanbevelen:
[https://lpdaac.usgs.gov/resources/e-learning/getting-started-gedi-l1b-data-python/](https://lpdaac.usgs.gov/resources/e-learning/getting-started-gedi-l1b-data-python/)

Inladen van de 'waveform' voor de geselecteerde shot:

In [ ]:
print('Dataset voor shot nummer: ', shot , 'is ingeladen als wvDF.')
wvDF = pd.read_csv('data/L1B_waveform_full.csv')

In [ ]:
# Laatste rij weghalen en relevante kolommen selecteren:
wvDF = wvDF[:-1][['rx_Waveform','rx_Elevation']]
wvDF

### Visualisatie van de ruwe 'waveform': 

In [ ]:
visL1B = hv.Curve(wvDF).opts(color='darkgreen', tools=['hover'], height=600, width=400,
                             xlim=(np.min(wvDF['rx_Waveform']) - 10, np.max(wvDF['rx_Waveform']) + 10), 
                             ylim=(np.min(wvDF['rx_Elevation']), np.max(wvDF['rx_Elevation'])),
                             fontsize={'xticks':10, 'yticks':10,'xlabel':16, 'ylabel': 16, 'title':13}, line_width=2.5, title=f'{str(shot)}')
visL1B

## Visualisatie van de ruwe waveform met de afgeleide RH-metrieken

In [ ]:
visL1B.opts(height=600, width=400, ylim=(np.min(rhShot), np.max(rhShot)+5), ylabel='Elevation (m)', xlabel='Amplitude (DN)') \
+ l2aVis.opts(height=600, width=400, ylim=(np.min(rhShot), np.max(rhShot)+5))


# GEDI Level 4 - INFO

Uit de GEDI- Level-2A data (meer specifiek de RH-metrieken) wordt een Level-4 product afgeleid. Deze bevat onder andere inschattingen voor de bovengrondse biomassa voor elke shot.

De GEDI-aanpak voor het ontwikkelen van AGBD-modellen met footprint houdt rekening met meerdere kandidaten, gestratificeerd naar wereldregio en PFT (Plant Functional Type) met verschillende functionele vormen. De modellen zijn ontwikkeld met behulp van een kwaliteitsgefilterde kalibratiedataset, bestaande uit **8.587 gesimuleerde golfvormen** in **21 landen**. Deze gegevens zijn bijgedragen door talrijke onderzoekers en gestandaardiseerd in de GEDI FSBD (Footprint Structural Biomass Database), die een levend gegevensarchief is dat in de loop der tijd groeit naarmate nieuwe datasets worden geassimileerd en verbeteringen worden aangebracht in bestaande gegevens.

Meer informatie over Level-4 data structuur:
-  [https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density.html#references](https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density.html#references)

-  pdf document: [Algorithm_theoretical_basis_document_for_GEDI_foot](Algorithm_theoretical_basis_document_for_GEDI_foot.pdf)

In wat volgt bekijken we hoe de Level-4 database er uit ziet, voor dezelfde shot als hierboven:

In [ ]:
# Start met data in te lezen
gedi_L4 = pd.read_csv('data/gedi_l4.csv')

In [ ]:
gedi_L4.columns.values

In [ ]:
# Enkel de shot selecteren:
gedi_L4_shot = gedi_L4[gedi_L4['shot_number'] == shot]
gedi_L4_shot

In [ ]:
# Informatie over het toegekende stratum
gedi_L4_shot['predict_stratum']

!['img/GEDI_L4_strata.png'](img/GEDI_L4_strata.png)

Beschrijving van de codes:
DBT, deciduous broadleaf trees), DNT (deciduous needleleaf trees), EBT (evergreen broadleaf trees), ENT (evergreen needleleaf trees), GSW (grasses, shrubs, and woodlands). Af (Africa), Au (Australia and Oceania), Eu (Europe), N-Am (North America north of southern Mexico), N-As (North Asia), S-Am (South America, Central America, southern Mexico, and the Caribbean), S-As (South Asia).

Uit de ATB (Algorithm Theoretical Basis), kunnen we vinden dat de gebruikte formule om tot de AGB-waarden voor het stratum `EBT_SA` te komen de volgende is:


Voor de shot, wordt de ingeschatte biomassa-densiteit(uitgedrukt in Mg/ha) dus:

In [ ]:
gedi_L4_shot['agbd']

> **Note**
> In deze versie van de AGB-algoritmen wordt er dus géén afzonderlijk stratum opgesteld voor het mangrove-ecosysteem. Er wordt wel gepleit om in toekomstige versies hiervoor meer rekening te houden, maar daarvoor is er meer data nodig!
> Terrestrial Laser Scans (TLS) voor mangrove kan hierin een rol spelen, om met een lagere foutenmarge GEDI-AGB waarden te berekenen

# Oefening: visualizatie van een rode mangrove shot

In voorgaande voorbeelden hebben we een GEDI-shot gevisualiseerd voor een locatie met jonge zwarte mangrove. In een tweede voorbeeld plotten we een shot binnen rode mangrove. Tracht hiervoor code te kopiëren uit voorgaande code!

<div class="alert alert-success">

**STAP 1 - Level 2A**:

Start met het vinden en visualiseren van het shot binnen de Level-2A dataset. Maak hierbij gebruik de code hierboven, waarbij je het shotnummer aanpast.

In [ ]:
# Shotnummer voor een shot binnen rode mangrove
shot_rm = 155570800200090851

<div class="alert alert-success">

**STAP 2 - Level 1B**:

Vervolgens kunnen we ook de ruwe waveformdata visualizeren. Hiervoor is de waveformdata gegeven:


In [ ]:
# Inladen van de ruwe waveformdata
GEDI_L1B = pd.read_csv('data/L1B_waveform_RM_full.csv')

<div class="alert alert-success">

**STAP 3 - Level 4 AGB data**:

Tot slot kunnen we op basis van de GEDI-L4 data een ingeschatte biomassa-hoeveelheid opvragen. Hiervoor kun je gebruik maken van de ```gedi_L4.csv```, waarbij je deze filter op basis van het nieuwe shotnummer. Wat is de ingeschatte biomassadensiteit? 

# EXTRA: visualiseren van alle shots (L2A)

Tot slot kunnen we de GEDI-metrieken ook voor elke shot gaan visualizeren, om zo een inzicht te krijgen in de structuur van het landschap.

## 1) Alle GEDI-shots extraheren

In het voorgaande bekeken we de relative hoogte van slechts één GEDI-shot. We kunnen nu ook de volledige GEDI-dataset extraheren en visualizeren

In [ ]:
beamNames = [g for g in gediL2A.keys() if g.startswith('BEAM')]

In [ ]:
beamNames

## 1) Subsetten van GEDI-data

We beschikken over 2 vector-datasets:
- De GEDI-shots (gediSDS)
- De ROI


In [ ]:
# Set up lists to store data
shotNum, dem, zElevation, zHigh, zLat, zLon, rh25, rh98, rh100 ,quality ,degrade, sensitivity ,beamI = ([] for i in range(13))  

In [ ]:
# Loop through each beam and open the SDS needed
for b in beamNames:
    [shotNum.append(h) for h in gediL2A[[g for g in gediSDS if g.endswith('/shot_number') and b in g][0]][()]]
    [dem.append(h) for h in gediL2A[[g for g in gediSDS if g.endswith('/digital_elevation_model') and b in g][0]][()]]
    [zElevation.append(h) for h in gediL2A[[g for g in gediSDS if g.endswith('/elev_lowestmode') and b in g][0]][()]]  
    [zHigh.append(h) for h in gediL2A[[g for g in gediSDS if g.endswith('/elev_highestreturn') and b in g][0]][()]]  
    [zLat.append(h) for h in gediL2A[[g for g in gediSDS if g.endswith('/lat_lowestmode') and b in g][0]][()]]  
    [zLon.append(h) for h in gediL2A[[g for g in gediSDS if g.endswith('/lon_lowestmode') and b in g][0]][()]]  
    [rh25.append(h[25]) for h in gediL2A[[g for g in gediSDS if g.endswith('/rh') and b in g][0]][()]]  
    [rh98.append(h[98]) for h in gediL2A[[g for g in gediSDS if g.endswith('/rh') and b in g][0]][()]]
    [rh100.append(h[100]) for h in gediL2A[[g for g in gediSDS if g.endswith('/rh') and b in g][0]][()]]  
    [quality.append(h) for h in gediL2A[[g for g in gediSDS if g.endswith('/quality_flag') and b in g][0]][()]]  
    [degrade.append(h) for h in gediL2A[[g for g in gediSDS if g.endswith('/degrade_flag') and b in g][0]][()]]  
    [sensitivity.append(h) for h in gediL2A[[g for g in gediSDS if g.endswith('/sensitivity') and b in g][0]][()]]  
    [beamI.append(h) for h in [b] * len(gediL2A[[g for g in gediSDS if g.endswith('/shot_number') and b in g][0]][()])]  

In [ ]:
# Convert lists to Pandas dataframe
allDF = pd.DataFrame({'Shot Number': shotNum, 'Beam': beamI, 'Latitude': zLat, 'Longitude': zLon, 'Tandem-X DEM': dem,
                      'Elevation (m)': zElevation, 'Canopy Elevation (m)': zHigh, 'Canopy Height (rh100)': rh100, 'RH 98': rh98,
                      'RH 25': rh25, 'Quality Flag': quality, 'Degrade Flag': degrade, 'Sensitivity': sensitivity})


In [ ]:
del beamI, degrade, dem, gediSDS, rh100, rh98, rh25, quality, sensitivity, zElevation, zHigh, zLat, zLon, shotNum


## Spatial Subsetting

We zijn enkel geïnteresseerd in de Mangrove-zone:

In [ ]:
#Spatiaal kader:
Mangrove_ROI.envelope[0].bounds

In [ ]:
minLon, minLat, maxLon, maxLat = Mangrove_ROI.envelope[0].bounds[:]  # Define the min/max lat/lon from the bounds of Redwood NP


In [ ]:
allDF = allDF.where(allDF['Latitude'] > minLat)
allDF = allDF.where(allDF['Latitude'] < maxLat)
allDF = allDF.where(allDF['Longitude'] > minLon)
allDF = allDF.where(allDF['Longitude'] < maxLon)

Resterende GEDI-shots om te visualiseren

In [ ]:
allDF = allDF.dropna()  # Drop shots outside of the ROI
len(allDF)

## Kwaliteitsfiltering

GEDI bevat zelf enkele 'kwaliteits'-indicatoren, waarop de dataset gefilterd kan worden. Hierbij worden de shot-opnames die vermoedelijk een lage kwaliteit hebben gefilterd uit de dataset, zodat enkel de bruikbare data overblijft

In [ ]:
# Set any poor quality returns to NaN
allDF = allDF.where(allDF['Quality Flag'].ne(0))
allDF = allDF.where(allDF['Degrade Flag'] < 1) 
allDF = allDF.where(allDF['Sensitivity'] > 0.95)
allDF = allDF.dropna()
print('Resterend aantal GEDI-shots na kwaliteitsfiltering: ', len(allDF))

In [ ]:
allDF['geometry'] = allDF.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)

In [ ]:
# Convert to geodataframe
allDF = gp.GeoDataFrame(allDF)
allDF = allDF.drop(columns=['Latitude','Longitude'])

In [ ]:
allDF['Shot Number'] = allDF['Shot Number'].astype(str)  # Convert shot number to string

vdims = []
for f in allDF:
    if f not in ['geometry']:
        vdims.append(f)

visual = pointVisual(allDF, vdims = vdims)
visual * gv.Polygons(Mangrove_ROI['geometry']).opts(line_color='red', color=None)

### Plot maken van de Canopy Height

In [ ]:
# Plot the basemap and geoviews Points, defining the color as the Canopy Height for each shot
(gvts.EsriImagery * gv.Points(allDF, vdims=vdims).options(color='Canopy Height (rh100)',cmap='plasma', size=3, tools=['hover'],
                                                          clim=(0,40), colorbar=True, clabel='Meters',
                                                          title='GEDI Canopy Height over Redwood National Park: June 19, 2019',
                                                          fontsize={'xticks': 10, 'yticks': 10, 'xlabel':16, 'clabel':12,
                                                                    'cticks':10,'title':16,'ylabel':16})).options(height=500,
                                                                                                                  width=900)